# Libraries

In [1]:
import sys
import os 
import time as time
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd 
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier,  GradientBoostingClassifier
import tensorflow as tf
import keras
import re
import time
from sklearn.preprocessing import LabelEncoder

Using TensorFlow backend.


# Read datasets 


In [2]:
tic = time.time()

app_train = pd.read_csv('application_train.csv')
app_test = pd.read_csv('application_test.csv')
bureau  = pd.read_csv('bureau.csv')
bureau_balance = pd.read_csv('bureau_balance.csv')
cred_bal       = pd.read_csv('credit_card_balance.csv')
install_pay    = pd.read_csv('installments_payments.csv')
pos_cash_bal   = pd.read_csv('POS_CASH_balance.csv')
previous_app   = pd.read_csv('previous_application.csv')


toc = time.time()

print("Czas procesowania %.3f" %(toc - tic)  )

Czas procesowania 47.108


# Data dimensions

In [3]:
print("Application test shape:  ", app_test.shape)
print("Application train shape: ", app_train.shape)
print("Bureau  shape: ",  bureau.shape)
print("BUreau  Balance: ", bureau_balance.shape)
print("Credit  Balance shape: ", cred_bal.shape )
print(" Installments Payments shape: ", install_pay.shape )
print(" Pos cash balance  shape:    ", pos_cash_bal.shape )
print(" Previous APplication shape : ", previous_app.shape)

Application test shape:   (48744, 121)
Application train shape:  (307511, 122)
Bureau  shape:  (1716428, 17)
BUreau  Balance:  (27299925, 3)
Credit  Balance shape:  (3840312, 23)
 Installments Payments shape:  (13605401, 8)
 Pos cash balance  shape:     (10001358, 8)
 Previous APplication shape :  (1670214, 37)


# Concatenate to one dataset

In [4]:
y_train = app_train.loc[:, ['TARGET', 'SK_ID_CURR']]
app_train = app_train.drop(['TARGET'] , axis = 1)

app_merged = pd.concat([app_train, app_test] )

# Numerical Variables to fillna

In [5]:

# DLa jakich zmiennych zrobic replace dla na
var_to_replace_nan = []

for i in  app_merged.columns:
    #print("Zmienna", i, "Posiada typ danych \n", app_merged.loc[:, i].dtypes)
    if app_merged.loc[:, i].dtypes in ('float64', 'int64'):
        var_to_replace_nan.append(str(i))


Replace na with some guess -9999 value

In [6]:
# Miss na for some absurd value


for i in var_to_replace_nan:
    
    app_merged.loc[:, i]  = app_merged.loc[:, i].fillna(-9999)
    app_merged.loc[:, i]   = app_merged.loc[:, i].fillna(-9999)
    
    #print("check:", i,  "\n", sum(np.isnan(app_merged.loc[:, i]))/app_merged.shape[0])

# Label Encoding


In [7]:
#LIczba unikalnych poziomow dla object dtype
var_to_label_enc = []

for i in app_merged.columns:
    #print("Badania dla", i)
    if app_merged.loc[:, i].dtypes == "object":
        #print(i, "To object app")
        var_to_label_enc.append(str(i))

In [8]:
for i in var_to_label_enc:
    lbl = LabelEncoder()
    app_merged.loc[:, i + str('_enc')] = lbl.fit_transform(app_merged[i].astype(str))
    #app_test.loc[:, i]  = lbl.fit_transform(app_train[i].fillna('NaN'))
    #print("poprawnie przeprowadzone dla \n", i)

Indexes - Train and test dataset

In [9]:
ind_train = app_merged.index[app_merged.SK_ID_CURR.isin(y_train.SK_ID_CURR) ]
ind_test  = app_merged.index[-ind_train]

# Aggregates 
- bureau credit
- previous app


Information about client previous behaviour. Dummy variable  if client has mortgage/loan/credit_card

In [147]:
app_merged.to_csv('app_merged.txt', sep = "|")

In [27]:
toc = time.time()


funs = ['sum', 'mean', 'std', 'median', 'max']

bureau_credit_activity = bureau.groupby(['SK_ID_CURR', 
                                         'CREDIT_ACTIVE']).agg({'SK_ID_CURR' : 'count',
                                         'CREDIT_ACTIVE' : 'count',
                                         'DAYS_CREDIT' : funs, 
                                         'AMT_CREDIT_MAX_OVERDUE' : funs ,
                                         'AMT_CREDIT_SUM'          : funs })



bureau_credit_type = bureau.groupby(['SK_ID_CURR',
                                        'CREDIT_TYPE']).agg({'SK_ID_CURR' : 'count',
                                        'CREDIT_TYPE' : 'count'         ,
                                        'DAYS_CREDIT'            : funs , 
                                        'AMT_CREDIT_MAX_OVERDUE' : funs , 
                                        'AMT_CREDIT_SUM'         : funs })


previous_app_contract_type = previous_app.groupby(['SK_ID_CURR', 
                                                  'NAME_CONTRACT_TYPE']).agg({'SK_ID_CURR' : 'count',
                                                  'NAME_CONTRACT_TYPE' : 'count' ,
                                                  'AMT_ANNUITY' : funs, 
                                                  'AMT_APPLICATION' : funs, 
                                                  'AMT_CREDIT'      : funs,
                                                  'AMT_DOWN_PAYMENT' : funs                     
})



previous_app_aggr = previous_app.groupby(['SK_ID_CURR']).agg({
    'SK_ID_CURR' : 'count', 
    'AMT_ANNUITY' : funs, 
    'AMT_APPLICATION' : funs, 
    'AMT_CREDIT'      : funs, 
    'AMT_DOWN_PAYMENT' : funs
})



client_has_mortgage = bureau[bureau.CREDIT_TYPE  == 'Mortgage']
client_has_mortgage['CLIENT_HAS_MORTGAGE'] = 1

client_has_cash_loan = previous_app.loc[previous_app.NAME_CONTRACT_TYPE == "Cash loans", ['SK_ID_CURR', 'NAME_CONTRACT_TYPE']]
client_has_cash_loan['CLIENT_HAS_CASH_LOAN'] = 1



client_has_consumer_loan = previous_app.loc[previous_app.NAME_CONTRACT_TYPE == 'Consumer loans', 
                                           ['SK_ID_CURR', 'NAME_CONTRACT_TYPE']]
client_has_consumer_loan['CLIENT_HAS_CONSUMER_LOAN'] = 1


client_has_mortgage = client_has_mortgage.loc[:, ['SK_ID_CURR', 'CLIENT_HAS_MORTGAGE'] ]


tic = time.time()

print("Processing time: \n %.3f" % (tic-toc))


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Processing time: 
 8.112


In [11]:
toc = time.time() 

app_merged = app_merged.merge(client_has_cash_loan, on = 'SK_ID_CURR', how = 'left' , suffixes= ['global', 'cash_loan'])
app_merged = app_merged.merge(client_has_consumer_loan, on = 'SK_ID_CURR', how = 'left', suffixes = ['global' ,'consumer_loan'])

app_merged =  app_merged.merge(client_has_mortgage, on = 'SK_ID_CURR', 
                               how = 'left' , suffixes = ['global', 'mortgages'] )


tic = time.time()

print("Processing time: \n %.3f" % (tic - toc))

Processing time: 
 9.136


In [12]:
app_merged.loc[:, ['CLIENT_HAS_CASH_LOAN', 'CLIENT_HAS_CONSUMER_LOAN', 'CLIENT_HAS_MORTGAGE']] = app_merged.loc[:, ['CLIENT_HAS_CASH_LOAN', 'CLIENT_HAS_CONSUMER_LOAN', 'CLIENT_HAS_MORTGAGE']].fillna(0)


In [22]:
bureau_credit_activity.columns

MultiIndex(levels=[['SK_ID_CURR', 'DAYS_CREDIT', 'AMT_CREDIT_MAX_OVERDUE', 'AMT_CREDIT_SUM'], ['count', 'max', 'mean', 'median', 'std', 'sum']],
           labels=[[0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3], [0, 5, 2, 4, 3, 1, 5, 2, 4, 3, 1, 5, 2, 4, 3, 1]])

In [66]:
bureau_credit_activity.columns

MultiIndex(levels=[['SK_ID_CURR', 'CREDIT_ACTIVE', 'DAYS_CREDIT', 'AMT_CREDIT_MAX_OVERDUE', 'AMT_CREDIT_SUM'], ['count', 'max', 'mean', 'median', 'std', 'sum']],
           labels=[[0, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4], [0, 0, 5, 2, 4, 3, 1, 5, 2, 4, 3, 1, 5, 2, 4, 3, 1]])

In [122]:
import re

In [103]:
test = bureau_credit_max_ovd

In [144]:
test['ind'] =  test.index

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [145]:
test['ind'] =  test['ind'].astype(str)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [146]:
test.dtypes

AMT_CREDIT_MAX_OVERDUE_sum       float64
AMT_CREDIT_MAX_OVERDUE_mean      float64
AMT_CREDIT_MAX_OVERDUE_std       float64
AMT_CREDIT_MAX_OVERDUE_median    float64
AMT_CREDIT_MAX_OVERDUE_max       float64
ind                               object
dtype: object

In [142]:
test['ind'] = test['ind'].replace(to_replace = ',', value = " ", inplace = True, regex = True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:4619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [143]:
test['ind']

SK_ID_CURR  CREDIT_ACTIVE
100001      Active           None
            Closed           None
100002      Active           None
            Closed           None
100003      Active           None
            Closed           None
100004      Closed           None
100005      Active           None
            Closed           None
100007      Closed           None
100008      Active           None
            Closed           None
100009      Active           None
            Closed           None
100010      Active           None
            Closed           None
100011      Closed           None
100013      Closed           None
100014      Active           None
            Closed           None
100015      Closed           None
100016      Active           None
            Closed           None
100017      Closed           None
100019      Active           None
100020      Active           None
            Closed           None
100022      Active           None
100023      Active    

In [60]:
bureau_activity_d_cred = bureau_credit_activity[ 'DAYS_CREDIT' ]
bureau_credit_max_ovd  = bureau_credit_activity[ 'AMT_CREDIT_MAX_OVERDUE' ]
bureau_amt_credit      = bureau_credit_activity[ 'AMT_CREDIT_SUM' ]

max_overdue_cols = [ "AMT_CREDIT_MAX_OVERDUE_" + str(i) for i in bureau_credit_activity[ 'AMT_CREDIT_MAX_OVERDUE' ].columns ]
amt_credit_cols  = [ "AMT_CREDIT_" + str(i) for i in bureau_credit_activity[ 'AMT_CREDIT_SUM' ].columns ]
days_credit_cols = [ "DAYS_CREDIT_"  + str(i) for i in bureau_credit_activity[ 'DAYS_CREDIT' ].columns ]

bureau_activity_d_cred.columns = days_credit_cols
bureau_credit_max_ovd.columns = max_overdue_cols 
bureau_amt_credit.columns = amt_credit_cols


In [ ]:

bureau_credit_activity['DAYS_CREDIT'].columns =  days_credit_cols


# Bureau activity joins to app_merged 


Index(['DAYS_CREDIT_sum', 'DAYS_CREDIT_mean', 'DAYS_CREDIT_std',
       'DAYS_CREDIT_median', 'DAYS_CREDIT_max'],
      dtype='object')

# Prototype - baseline model check

- Cross Validation
- Stratified K fold

In [ ]:
app_train_enc = app_merged.loc[:max(ind_train), :]
app_test_enc = app_merged.loc[max(ind_train):, :]

In [ ]:
app_train_enc = app_train_enc.drop(var_to_label_enc, axis = 1)

In [ ]:
rf = RandomForestClassifier()


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_valid, y_tr, y_val = train_test_split(app_train_enc, y_train.TARGET,  
                                                     random_state = 321, 
                                                     test_size = .3)

In [ ]:
print("Treningowy zestaw danych", X_train.shape[1])
print("Walidacyjny zestaw danych", X_valid.shape[1])

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve

In [ ]:
model = rf.fit(X_train, y_tr)


In [ ]:
test_pred_proba = model.predict_proba(X_valid)[:, 0]
test_pred   = model.predict(X_valid)

In [ ]:
pd.crosstab(test_pred, columns = 'count')/len(test_pred)

In [ ]:
roc_auc_score(test_pred, y_val)

In [ ]:
X_train = X_train.reset_index()
y_tr = y_tr.reset_index()

In [ ]:
from sklearn.model_selection import StratifiedKFold, KFold

kfold = KFold(n_splits = 10, 
                       random_state = 321).split(X_train, y_tr.TARGET)

In [ ]:
from sklearn.metrics import roc_curve

In [ ]:
scores = []

for k, (train, valid) in enumerate(kfold):
    print(max(train), min(train))
    print("Valid \n", max(valid), min(valid))
    rf.fit(X_train.loc[train, :], y_tr.TARGET[train] )
    pred = rf.predict(X_train.loc[valid, :])
    score = roc_auc_score(pred , y_tr.TARGET[valid])
    scores.append(score)
    print('Fold: %s, Class dist.: %s, Acc: %.3f' % (k+1,
          np.bincount(y_tr.TARGET[train]), score))

In [ ]:

kfold = KFold(n_splits = 10, 
                       random_state = 321).split(X_train, y_tr.TARGET)

In [ ]:
from sklearn.ensemble  import AdaBoostClassifier


ada_fit = AdaBoostClassifier()


scores_ada = []

for k, (train, valid) in enumerate(kfold):
    print(max(train), min(train))
    print("Valid \n", max(valid), min(valid))
    ada_fit.fit(X_train.loc[train, :], y_tr.TARGET[train] )
    pred = ada_fit.predict(X_train.loc[valid, :])
    score = roc_auc_score(pred , y_tr.TARGET[valid])
    scores_ada.append(score)
    print('Fold: %s, Class dist.: %s, Acc: %.3f' % (k+1,
          np.bincount(y_tr.TARGET[train]), score))

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gbm = GradientBoostingClassifier()

kfold = KFold(n_splits = 10, random_state = 321).split(X_train, y_tr.TARGET)

In [ ]:


scores_gbm = []

for k, (train, valid) in enumerate(kfold):
    print(max(train), min(train))
    print("Valid \n", max(valid), min(valid))
    gbm.fit(X_train.loc[train, :], y_tr.TARGET[train] )
    pred = gbm.predict(X_train.loc[valid, :])
    score = roc_auc_score(pred , y_tr.TARGET[valid])
    scores_gbm.append(score)
    print('Fold: %s, Class dist.: %s, Acc: %.3f' % (k+1,
          np.bincount(y_tr.TARGET[train]), score))